In [1]:
%pip install pandas matplotlib
%pip install scikit-learn

You should consider upgrading via the '/Users/joelsng/.pyenv/versions/3.9.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/joelsng/.pyenv/versions/3.9.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install xgboost==2.0.3

You should consider upgrading via the '/Users/joelsng/.pyenv/versions/3.9.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import re, json
from pathlib import Path
import datetime
import xgboost as xgb
import numpy as np
import pandas as pd
import json

In [4]:
df = pd.read_csv("sampleData/master_orders.csv")
df["Order Date"] = pd.to_datetime(df["Order Date"], utc=True)
df = df.sort_values("Order Date")

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 18217 entries, 18096 to 2218
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Order Date     18004 non-null  datetime64[ns, UTC]
 1   SKU            17015 non-null  object             
 2   Item           18216 non-null  object             
 3   Variant        14849 non-null  object             
 4   Quantity       18177 non-null  float64            
 5   Value          12212 non-null  object             
 6   Delivery Date  12317 non-null  object             
 7   Delivery Time  12317 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(6)
memory usage: 1.3+ MB
None


# Aggregate based on monthly

In [6]:
monthly = (
    df
    .set_index("Order Date")
    .groupby(["SKU", pd.Grouper(freq="ME")])["Quantity"]
    .sum()
    .reset_index()
    .rename(columns={"Order Date": "Month"})
    .sort_values(["SKU", "Month"])
)

In [7]:
monthly.groupby("Month")["Quantity"].sum().reset_index().sort_values("Month")

,Month,Quantity
0,2022-04-30 00:00:00+00:00,91.0
1,2022-05-31 00:00:00+00:00,280.0
2,2022-06-30 00:00:00+00:00,339.0
3,2022-07-31 00:00:00+00:00,431.0
4,2022-08-31 00:00:00+00:00,392.0
5,2022-09-30 00:00:00+00:00,534.0
6,2022-10-31 00:00:00+00:00,412.0
7,2022-11-30 00:00:00+00:00,704.0
8,2022-12-31 00:00:00+00:00,720.0
9,2023-01-31 00:00:00+00:00,502.0


# Get top 5 SKUs

In [8]:
top_k = 0

In [9]:
vol_df = (
    monthly.groupby("SKU", observed=True)["Quantity"]
    .sum()
    .rename("total_qty")
    .reset_index()
)

vol_df = vol_df.sort_values(["total_qty", "SKU"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

vol_df.head()

,SKU,total_qty
0,SF50,5108.0
1,CU-1,1423.0
2,SC5,688.0
3,DR50,592.0
4,SC15,546.0


In [10]:
top5 = vol_df.head(top_k)["SKU"].tolist()
top5

[]

# Stratify SKUs into sales volume

In [11]:
rest_df = vol_df[~vol_df["SKU"].isin(top5)].copy()
rest_total = rest_df["total_qty"].sum()

rest_df["cum_share"] = rest_df["total_qty"].cumsum() / rest_total


In [12]:
bins   = [-np.inf, 0.70, 0.95, np.inf]
labels = ["highest", "medium", "rest"]

rest_tiers = pd.cut(
    rest_df["cum_share"],
    bins=bins,
    labels=labels,
    include_lowest=True
).astype("string")

In [13]:
solo_labels = pd.Series({sku: f"solo::{sku}"} for sku in top5)
solo_labels = pd.Series({sku: f"solo::{sku}" for sku in top5}, name="tier", dtype="string")

rest_labels = pd.Series(rest_tiers.values, index=rest_df["SKU"].values, name="tier", dtype="string")

sku_tier = pd.concat([solo_labels, rest_labels])
sku_tier = sku_tier.astype("string")

sku_tier.name = "tier"


In [14]:
tiered = monthly.merge(sku_tier.to_frame(), left_on="SKU", right_index=True, how="inner")


In [15]:
print(sku_tier.value_counts())
print((sku_tier.value_counts(normalize=True) * 100).round(2).astype(str) + "%")

tier
rest       920
medium     545
highest     40
Name: count, dtype: Int64
tier
rest       61.13%
medium     36.21%
highest     2.66%
Name: proportion, dtype: object


In [16]:
tiered

,SKU,Month,Quantity,tier
0,SI1623T/MapleWH,2025-03-31 00:00:00+00:00,1.0,rest
1,**SB_MU_120135M_3036 (SR4/3036)-Add 4 shelves,2025-10-31 00:00:00+00:00,1.0,rest
2,*CTM11971/Soft close/Black pvc 10cm,2025-09-30 00:00:00+00:00,1.0,rest
3,*CTM11975-soft close slider,2025-09-30 00:00:00+00:00,1.0,rest
4,"*DC1289/3045-NOLEGS, HDL172-128-SS",2025-08-31 00:00:00+00:00,1.0,rest
...,...,...,...,...
3985,Zara/Storage/SuperSingle,2025-04-30 00:00:00+00:00,1.0,rest
3986,Zara/Storage/SuperSingle,2025-05-31 00:00:00+00:00,1.0,rest
3987,c,2022-11-30 00:00:00+00:00,1.0,rest
3988,metal legs,2025-09-30 00:00:00+00:00,1.0,rest


In [17]:
df_tier_month = (
    tiered.assign(Month=pd.to_datetime(tiered["Month"]).dt.to_period("M").dt.to_timestamp())
      .groupby(["tier", "Month"], as_index=False)["Quantity"]
      .sum()
      .sort_values(["tier", "Month"])
)

df_tier_month

/var/folders/tk/ptln5ljx66g1yqhg_dd36_200000gn/T/ipykernel_22353/1324460882.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  tiered.assign(Month=pd.to_datetime(tiered["Month"]).dt.to_period("M").dt.to_timestamp())


,tier,Month,Quantity
0,highest,2022-04-01,33.0
1,highest,2022-05-01,146.0
2,highest,2022-06-01,218.0
3,highest,2022-07-01,273.0
4,highest,2022-08-01,227.0
...,...,...,...
124,rest,2025-06-01,75.0
125,rest,2025-07-01,125.0
126,rest,2025-08-01,88.0
127,rest,2025-09-01,106.0


# Feature Engineering

In [18]:
df_tier_month["month_num"] = df_tier_month["Month"].dt.month
df_tier_month["year"] = df_tier_month["Month"].dt.year

# demand lag by x months
df_tier_month = df_tier_month.sort_values(["tier", "Month"])
df_tier_month["lag_1"] = df_tier_month.groupby("tier")["Quantity"].shift(1)
df_tier_month["lag_2"] = df_tier_month.groupby("tier")["Quantity"].shift(2)
df_tier_month["lag_3"] = df_tier_month.groupby("tier")["Quantity"].shift(3)

# month sin/cos seasonality
df_tier_month["month_sin"] = np.sin(2 * np.pi * df_tier_month["month_num"] / 12)
df_tier_month["month_cos"] = np.cos(2 * np.pi * df_tier_month["month_num"] / 12)


In [19]:
# use log_qty as target to train
df_tier_month["log_qty"] = np.log1p(df_tier_month["Quantity"])

In [20]:
df_tier_month

,tier,Month,Quantity,month_num,year,lag_1,lag_2,lag_3,month_sin,month_cos,log_qty
0,highest,2022-04-01,33.0,4,2022,NaN,NaN,NaN,8.660254e-01,-5.000000e-01,3.526361
1,highest,2022-05-01,146.0,5,2022,33.0,NaN,NaN,5.000000e-01,-8.660254e-01,4.990433
2,highest,2022-06-01,218.0,6,2022,146.0,33.0,NaN,1.224647e-16,-1.000000e+00,5.389072
3,highest,2022-07-01,273.0,7,2022,218.0,146.0,33.0,-5.000000e-01,-8.660254e-01,5.613128
4,highest,2022-08-01,227.0,8,2022,273.0,218.0,146.0,-8.660254e-01,-5.000000e-01,5.429346
...,...,...,...,...,...,...,...,...,...,...,...
124,rest,2025-06-01,75.0,6,2025,37.0,20.0,52.0,1.224647e-16,-1.000000e+00,4.330733
125,rest,2025-07-01,125.0,7,2025,75.0,37.0,20.0,-5.000000e-01,-8.660254e-01,4.836282
126,rest,2025-08-01,88.0,8,2025,125.0,75.0,37.0,-8.660254e-01,-5.000000e-01,4.488636
127,rest,2025-09-01,106.0,9,2025,88.0,125.0,75.0,-1.000000e+00,-1.836970e-16,4.672829


# Training and Evaluation

In [21]:
RANDOM_SEED = 42
N_ESTIMATORS = 500
EARLY_STOPPING = 50

In [22]:
def train_xgb_core_api(
    df_tier,
    # feat=("month_num", "year", "sku_id", "lag_1", "lag_2", "lag_3", "month_sin", "month_cos"),
    feat=("month_num", "year", "lag_1", "lag_2", "lag_3", "month_sin", "month_cos"),
):
    # Drop rows that cannot form lag_1
    df_tier = df_tier.dropna(subset=["lag_1"]).reset_index(drop=True)

    # If too small, return 6-tuple with features so callers can unpack safely
    if len(df_tier) < 20:
        return None, None, None, None, None, list(feat)

    X = df_tier[list(feat)].to_numpy(dtype=float)
    y = df_tier["log_qty"].to_numpy(dtype=float)

    # 80/20 split with guards so validation is never empty
    split_idx = int(len(df_tier) * 0.8)
    if split_idx <= 0:
        split_idx = 1
    if split_idx >= len(df_tier):
        split_idx = len(df_tier) - 1

    X_tr, X_va = X[:split_idx], X[split_idx:]
    y_tr, y_va = y[:split_idx], y[split_idx:]

    dtr = xgb.DMatrix(X_tr, label=y_tr, feature_names=list(feat))
    dva = xgb.DMatrix(X_va, label=y_va, feature_names=list(feat))

    params = {
        "objective": "reg:squarederror",
        "eta": 0.1,
        "max_depth": 4,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "eval_metric": ["rmse", "mae"],
        "seed": 42,
    }
    ev = [(dtr, "train"), (dva, "valid")]
    bst = xgb.train(
        params,
        dtr,
        num_boost_round=N_ESTIMATORS,
        evals=ev,
        early_stopping_rounds=EARLY_STOPPING,
        verbose_eval=False,
    )

    # Predict using best iteration if available
    best_iter = getattr(bst, "best_iteration", None)
    if best_iter is None:
        pred = bst.predict(dva)
    else:
        pred = bst.predict(dva, iteration_range=(0, best_iter + 1))

    mae = float(np.mean(np.abs(y_va - pred)))
    rmse = float(np.sqrt(np.mean((y_va - pred) ** 2)))
    mape = float(np.mean(np.abs((y_va - pred) / np.clip(y_va, 1e-8, None))) * 100)
    smape = float(100 * np.mean(2 * np.abs(y_va - pred) / (np.abs(y_va) + np.abs(pred) + 1e-8)))

    return bst, mae, rmse, mape, smape, list(feat)


In [23]:
def slugify_tier(tier: str) -> str:
    s = re.sub(r"[^A-Za-z0-9]+", "_", str(tier)).strip("_")
    return s or "tier"

In [24]:
models = {}
tier_metrics = {}
report_rows = []
future_all = []

cur_dir = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd()
save_dir = cur_dir / "models"
save_dir.mkdir(parents=True, exist_ok=True)

tier_list = df_tier_month["tier"].dropna().astype(str).unique().tolist()

for tier in tier_list:
    sub = df_tier_month.loc[df_tier_month["tier"] == tier].copy()
    if sub.empty:
        print(f"[{tier}] skipped (no rows)")
        continue

    bst, mae, rmse, mape, smape, feat = train_xgb_core_api(sub)
    if bst is None:
        print(f"[{tier}] skipped (not enough data)")
        continue

    # Stable, safe filename per tier (e.g., solo__SKU_123)
    tier_slug = slugify_tier(tier)
    model_fname = f"xgb_sku_{tier_slug}_model.ubj"
    model_path = save_dir / model_fname
    bst.save_model(model_path)
    models[tier] = (bst, feat)

    # best_iteration can be None if early stopping not triggered
    best_iter = getattr(bst, "best_iteration", None)
    if best_iter is None:
        print(f"[{tier}] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%")
    else:
        print(f"[{tier}] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%  (best_iter={best_iter})")

    # ---- NEW: collect metrics for meta/report ----
    tier_metrics[tier] = {
        "model_path": model_fname,
        "features": list(feat),
        "mae": float(mae),
        "rmse": float(rmse),
        "mape_pct": float(mape),
        "smape_pct": float(smape),
        "best_iter": int(best_iter) if best_iter is not None else None,
        "notes": "Tiered SKU forecaster; 'solo::SKU' denotes per-SKU model."
    }
    report_rows.append({
        "tier": tier,
        "model_path": model_fname,
        "mae": float(mae),
        "rmse": float(rmse),
        "mape_pct": float(mape),
        "smape_pct": float(smape),
        "best_iter": int(best_iter) if best_iter is not None else None,
    })

    # ---------- Build next-month features per SKU in this tier ----------
    tier_series = (
        sub.loc[:, ["tier", "Month", "Quantity"]]
        .assign(Month=pd.to_datetime(sub["Month"]).dt.to_period("M").dt.to_timestamp())
        .groupby(["tier", "Month"], as_index=False)["Quantity"].sum()
        .sort_values(["tier", "Month"])
    )

    # lags over the TIER series
    lag_1 = tier_series["Quantity"].iloc[-1] if len(tier_series) >= 1 else np.nan
    lag_2 = tier_series["Quantity"].iloc[-2] if len(tier_series) >= 2 else np.nan
    lag_3 = tier_series["Quantity"].iloc[-3] if len(tier_series) >= 3 else np.nan

    fut_month = tier_series["Month"].iloc[-1] + pd.offsets.MonthEnd(1)
    fut_month_num = fut_month.month

    future = pd.DataFrame({
        "month_num": [fut_month_num],
        "year": [fut_month.year],
        "lag_1": [lag_1],
        "lag_2": [lag_2],
        "lag_3": [lag_3],
        "month_sin": [np.sin(2 * np.pi * fut_month_num / 12)],
        "month_cos": [np.cos(2 * np.pi * fut_month_num / 12)],
    })

    # Ensure exact feature order expected by this tier’s model
    # Ensure the feature matrix matches the model's expected columns and order
    for c in feat:
        if c not in future.columns:
            future[c] = np.nan
    X_future = future[list(feat)].to_numpy(dtype=float)

    dfut = xgb.DMatrix(X_future, feature_names=list(feat))

    preds_log = (
        models[tier][0].predict(dfut)
        if best_iter is None
        else models[tier][0].predict(dfut, iteration_range=(0, best_iter + 1))
    )
    preds = np.expm1(preds_log)  # inverse of log1p

    # One row per TIER only (no SKU column here)
    out = pd.DataFrame({
        "tier": [tier],
        "Month": [fut_month],
        "predicted_qty": preds
    })
    future_all.append(out)


future_preds = pd.concat(future_all, ignore_index=True)
print(future_preds.head(20))


[highest] MAE=0.620  RMSE=0.741  MAPE=13.17%  SMAPE=12.00%  (best_iter=50)
[medium] MAE=0.423  RMSE=0.477  MAPE=9.19%  SMAPE=8.95%  (best_iter=2)
[rest] MAE=0.348  RMSE=0.531  MAPE=9.57%  SMAPE=8.64%  (best_iter=226)
      tier      Month  predicted_qty
0  highest 2025-10-31     254.106674
1   medium 2025-10-31     112.631546
2     rest 2025-10-31      82.629784


In [25]:
# writing metadata artifacts
# models_meta.json
models_meta = {}
for t, info in tier_metrics.items():
    models_meta[t] = {
        "model_path": info["model_path"],
        "features": info["features"],
        "metrics": {
            "mae": info["mae"],
            "rmse": info["rmse"],
            "mape_pct": info["mape_pct"],
            "smape_pct": info["smape_pct"],
            "best_iter": info["best_iter"],
        },
        "notes": info["notes"],
    }

# # embed training meta
# models_meta["_training_context"] = {
#     "created_at": datetime.datetime.utcnow().isoformat() + "Z",
#     "xgboost_version": xgb.__version__,
# }

# with open(save_dir / "models_meta.json", "w") as f:
#     json.dump(models_meta, f, indent=2)
# print("Wrote:", save_dir / "models_meta.json")

# # sku_tier_map.csv & sku_tier_map.json (SKU → tier used)
# # Expect sku_tier: pd.Series where index=SKU, value=tier
# sku_tier_path_csv = save_dir / "sku_tier_map.csv"
# sku_tier_path_json = save_dir / "sku_tier_map.json"

# sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_csv(sku_tier_path_csv, index=False)
# sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_json(sku_tier_path_json, orient="records")
# print("Wrote:", sku_tier_path_csv)
# print("Wrote:", sku_tier_path_json)

# # human-readable report
# pd.DataFrame(report_rows).sort_values("tier").to_csv(save_dir / "models_report.csv", index=False)
# print("Wrote:", save_dir / "models_report.csv")

# Create an Inventory level model

In [26]:
# params
RANDOM_SEED = 42
N_ESTIMATORS = 500
EARLY_STOPPING = 50
MODEL_NAME = "xgb_inventory_model.ubj"   # single pooled model
ARTIFACT_DIR = (Path.cwd() / "models")  # adjust if needed
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

In [27]:
inventory_monthly = (
    monthly
    .groupby("Month")["Quantity"]
    .sum()
    .reset_index()
    .sort_values("Month")
    .reset_index(drop=True)
)

In [28]:
# Lag features
inventory_monthly["lag_1"] = inventory_monthly["Quantity"].shift(1)
inventory_monthly["lag_2"] = inventory_monthly["Quantity"].shift(2)
inventory_monthly["lag_3"] = inventory_monthly["Quantity"].shift(3)

# Seasonality
inventory_monthly["month_num"] = inventory_monthly["Month"].dt.month
inventory_monthly["year"] = inventory_monthly["Month"].dt.year
inventory_monthly["month_sin"] = np.sin(2 * np.pi * inventory_monthly["month_num"] / 12)
inventory_monthly["month_cos"] = np.cos(2 * np.pi * inventory_monthly["month_num"] / 12)

inventory_monthly["log_qty"] = np.log1p(inventory_monthly["Quantity"])

# Drop first few rows without lag features
# train_df = inventory_monthly.dropna(subset=["lag_1"]).reset_index(drop=True)

In [29]:
FEATURES = ["month_num", "year", "lag_1", "lag_2", "lag_3", "month_sin", "month_cos"]

In [30]:
inventory_monthly

,Month,Quantity,lag_1,lag_2,lag_3,month_num,year,month_sin,month_cos,log_qty
0,2022-04-30 00:00:00+00:00,91.0,NaN,NaN,NaN,4,2022,8.660254e-01,-5.000000e-01,4.521789
1,2022-05-31 00:00:00+00:00,280.0,91.0,NaN,NaN,5,2022,5.000000e-01,-8.660254e-01,5.638355
2,2022-06-30 00:00:00+00:00,339.0,280.0,91.0,NaN,6,2022,1.224647e-16,-1.000000e+00,5.828946
3,2022-07-31 00:00:00+00:00,431.0,339.0,280.0,91.0,7,2022,-5.000000e-01,-8.660254e-01,6.068426
4,2022-08-31 00:00:00+00:00,392.0,431.0,339.0,280.0,8,2022,-8.660254e-01,-5.000000e-01,5.973810
5,2022-09-30 00:00:00+00:00,534.0,392.0,431.0,339.0,9,2022,-1.000000e+00,-1.836970e-16,6.282267
6,2022-10-31 00:00:00+00:00,412.0,534.0,392.0,431.0,10,2022,-8.660254e-01,5.000000e-01,6.023448
7,2022-11-30 00:00:00+00:00,704.0,412.0,534.0,392.0,11,2022,-5.000000e-01,8.660254e-01,6.558198
8,2022-12-31 00:00:00+00:00,720.0,704.0,412.0,534.0,12,2022,-2.449294e-16,1.000000e+00,6.580639
9,2023-01-31 00:00:00+00:00,502.0,720.0,704.0,412.0,1,2023,5.000000e-01,8.660254e-01,6.220590


In [31]:
bst, mae, rmse, mape, smape, feat = train_xgb_core_api(inventory_monthly)

In [41]:
print( mae, rmse, mape, smape)

0.2755596405917108 0.3298890865869828 4.749634349115159 4.704244231997133


In [32]:
# Stable, safe filename
model_fname = MODEL_NAME
model_path = save_dir / model_fname
bst.save_model(model_path)
models["inventory"] = (bst, feat)

best_iter = getattr(bst, "best_iteration", None)
if best_iter is None:
    print(f"[inventory] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%")
else:
    print(f"[inventory] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%  (best_iter={best_iter})")

tier_metrics["inventory"] = {
    "model_path": model_fname,
    "features": list(feat),
    "mae": float(mae),
    "rmse": float(rmse),
    "mape_pct": float(mape),
    "smape_pct": float(smape),
    "best_iter": int(best_iter) if best_iter is not None else None,
    "notes": "Inventory overall level forecaster."
}
report_rows.append({
    "tier": "inventory",
    "model_path": model_fname,
    "mae": float(mae),
    "rmse": float(rmse),
    "mape_pct": float(mape),
    "smape_pct": float(smape),
    "best_iter": int(best_iter) if best_iter is not None else None,
})

[inventory] MAE=0.276  RMSE=0.330  MAPE=4.75%  SMAPE=4.70%  (best_iter=42)


In [33]:
last = inventory_monthly.sort_values("Month").iloc[-1]
fut_month = last["Month"] + pd.offsets.MonthEnd(1)
fut_m = fut_month.month

row = {
    "month_num": fut_m,
    "year": fut_month.year,
    "lag_1": last["Quantity"],
    "lag_2": last.get("lag_1", np.nan),
    "lag_3": last.get("lag_2", np.nan),
    "month_sin": np.sin(2 * np.pi * fut_m / 12),
    "month_cos": np.cos(2 * np.pi * fut_m / 12),
}

X = np.array([[row[f] for f in FEATURES]], dtype=float)
dfut = xgb.DMatrix(X, feature_names=FEATURES)

inv_forecast_log = float(bst.predict(dfut)[0])
inv_forecast = float(np.expm1(inv_forecast_log))

print(f"Next-month inventory forecast: {inv_forecast:.2f}")


Next-month inventory forecast: 349.48


In [34]:
for t, info in tier_metrics.items():
    print(t)

highest
medium
rest
inventory


In [35]:
for t, info in tier_metrics.items():
    if t == 'inventory':
        models_meta[t] = {
            "model_path": info["model_path"],
            "features": info["features"],
            "metrics": {
                "mae": info["mae"],
                "rmse": info["rmse"],
                "mape_pct": info["mape_pct"],
                "smape_pct": info["smape_pct"],
                "best_iter": info["best_iter"],
            },
            "notes": info["notes"],
        }

In [36]:
# embed training meta
models_meta["_training_context"] = {
    "created_at": datetime.datetime.utcnow().isoformat() + "Z",
    "xgboost_version": xgb.__version__,
}

with open(save_dir / "models_meta.json", "w") as f:
    json.dump(models_meta, f, indent=2)
print("Wrote:", save_dir / "models_meta.json")

# sku_tier_map.csv & sku_tier_map.json (SKU → tier used)
# Expect sku_tier: pd.Series where index=SKU, value=tier
sku_tier_path_csv = save_dir / "sku_tier_map.csv"
sku_tier_path_json = save_dir / "sku_tier_map.json"

sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_csv(sku_tier_path_csv, index=False)
sku_tier.to_frame().reset_index().rename(columns={"index": "SKU"}).to_json(sku_tier_path_json, orient="records")
print("Wrote:", sku_tier_path_csv)
print("Wrote:", sku_tier_path_json)

# human-readable report
pd.DataFrame(report_rows).sort_values("tier").to_csv(save_dir / "models_report.csv", index=False)
print("Wrote:", save_dir / "models_report.csv")

Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/models_meta.json
Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/sku_tier_map.csv
Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/sku_tier_map.json
Wrote: /Users/joelsng/Documents/GitHub/levelsliving-IMS/server/forecasting/models/models_report.csv


In [37]:
inventory_monthly.columns

Index(['Month', 'Quantity', 'lag_1', 'lag_2', 'lag_3', 'month_num', 'year',
       'month_sin', 'month_cos', 'log_qty'],
      dtype='object')

In [38]:
inventory_monthly.head()

,Month,Quantity,lag_1,lag_2,lag_3,month_num,year,month_sin,month_cos,log_qty
0,2022-04-30 00:00:00+00:00,91.0,NaN,NaN,NaN,4,2022,8.660254e-01,-0.500000,4.521789
1,2022-05-31 00:00:00+00:00,280.0,91.0,NaN,NaN,5,2022,5.000000e-01,-0.866025,5.638355
2,2022-06-30 00:00:00+00:00,339.0,280.0,91.0,NaN,6,2022,1.224647e-16,-1.000000,5.828946
3,2022-07-31 00:00:00+00:00,431.0,339.0,280.0,91.0,7,2022,-5.000000e-01,-0.866025,6.068426
4,2022-08-31 00:00:00+00:00,392.0,431.0,339.0,280.0,8,2022,-8.660254e-01,-0.500000,5.973810
